### The benchmark of SARCOS dataset

The performance of different algorithms on this dataset can be found in: https://github.com/Kaixhin/SARCOS

#### Ridge regression with fourier(polynomial()) feature: 2.111,  
NN feature not working.

In [24]:
from jylearn.data.reg_data import robot_inv_data
import numpy as np
import torch as th
device = "cuda" if th.cuda.is_available() else "cpu"

Loss = th.nn.MSELoss()
X_train, Y_train, X_val, Y_val, X_test, Y_test = robot_inv_data()

In [25]:
X_train.shape

(40000, 21)

In [26]:
X, Y = np.concatenate([X_train, X_val], axis=0), np.concatenate([Y_train, Y_val], axis=0)
from jylearn.feature.polynomial import PolynomialFT
from jylearn.feature.bellcurve import BellCurve
from jylearn.feature.fourier import FourierBases
from jylearn.parametric.ridge import RidgeReg

param1 = np.linspace(0.05, 0.99, 7)
param2 = np.arange(1, 7)

f1 = FourierBases(param1, param2)
f2 = PolynomialFT(2)

X_f = f1(f2(X))
X_test_f = f1(f2(X_test))
print("feature dim: ", X_f.shape[1])

feature dim:  6805


In [27]:
X_f_t, Y_t, X_test_f_t, Y_t_test = th.from_numpy(X_f).to(device), th.from_numpy(Y).to(device), th.from_numpy(X_test_f).to(device), th.from_numpy(Y_test).to(device)
rr = RidgeReg().fit(X_f_t, Y_t)
pred = rr.predict(X_test_f_t)
mse = Loss(pred, Y_t_test)
print(mse)

100%|██████████| 10/10 [01:29<00:00,  8.92s/it]


In [3]:
X_train, Y_train, X_val, Y_val, X_test, Y_test =\
    th.from_numpy(X_train).to(device), th.from_numpy(Y_train).to(device), th.from_numpy(X_val).to(device), \
    th.from_numpy(Y_val).to(device), th.from_numpy(X_test).to(device), th.from_numpy(Y_test).to(device)
    
from torch.optim import Adam
from torch.nn import MSELoss
from jylearn.parametric.mlp import MLP

#### Neural network: 1.469

In [ ]:
param = {"layer":4, "nodes":[21, 500, 500, 7], "batch":128, "lr":1e-3, "decay":0.}
net = MLP(param).to(device)
print(net)
Loss = MSELoss()
parameters = MLP.setParams(net, param["decay"])
optimizer = Adam(parameters, lr=param["lr"])
for _ in range(100):
    for i in range(len(X_train)//param["batch"]):
        optimizer.zero_grad()
        index = th.randperm(len(X_train))
        curr_index = index[i*param["batch"]:(i+1)*param["batch"]]
        X_b = X_train[curr_index]
        Y_b = Y_train[curr_index]
        pred = net(X_b)
        L = Loss(pred, Y_b)
        L.backward()
        optimizer.step()
    with th.no_grad():
        pred_val = net(X_val)
        L_val = Loss(pred_val, Y_val)
        print("Curr validation loss: ", L_val)
net.eval()
pred_test = net(X_test)
print(Loss(pred_test, Y_test))

#### subsample GPR without hyperparameters optimization:

In [10]:
from jylearn.nonparametric.gpr import ExactGPR
from jylearn.kernel.kernels import RQK, White

In [11]:
X_test, Y_test = X_test.double(), Y_test.double()
perm = th.randperm(X_train.size(0))
idx = perm[:4000]
X_train_, Y_train_ = X_train[idx].double(), Y_train[idx].double()

In [18]:
l = np.ones(21,) * 10
kernel = White(dim=21, c=4.) + RQK(dim=21, l=l, alpha=4., sigma=1.)
gpr = ExactGPR(kernel=kernel)
gpr.fit(X_train_, Y_train_, call_hyper_opt=True, lr=4e-3, epoch=10)

-7203.02: 100%|██████████| 3000/3000 [19:14<00:00,  2.60it/s] 

The evidence is:  tensor(-7202.5051, device='cuda:0', dtype=torch.float64)


In [21]:
pred = gpr.predict(X_test)
print(Loss(pred, Y_test))
del gpr, pred, kernel

th.cuda.empty_cache()

tensor(34.1621, device='cuda:0', dtype=torch.float64)
